In [21]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [22]:
from selenium import webdriver
from selenium.common import exceptions
import re
import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 크롤링

In [23]:
OUTPUT_FILE_NAME = 'output.txt'

In [24]:
wd = "D:\\Users\\kimdu\\Desktop\\GRAD\\chromedriver.exe"

In [25]:
import urllib.request
def get_text(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = ''
    for item in soup.find_all('div', id='articeBody'):
        text = text + str(item.find_all(text=True))
    return text

In [26]:
def clean_text(text):
    cleaned_text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\.]+", " ", text)
    return cleaned_text

In [27]:
def get_title(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = soup.select_one('h2.end_tit')
    text_r = text.get_text()
    return text_r

In [28]:
def comment(url):
    wd = "D:\\Users\\kimdu\\Desktop\\GRAD\\chromedriver.exe"
    driver = webdriver.Chrome(wd)
    driver.get(url)
    pages = 0
    try:
        while True:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#cbox_module > div > div.u_cbox_paginate > a"))).click()
            time.sleep(1.5)
            print(pages, end=" ")
            pages+=1
    except exceptions.ElementNotVisibleException as e: # 페이지 끝
        pass
    
    except Exception as e: # 다른 예외 발생시 확인
        print(e)
    html = driver.page_source
    dom = BeautifulSoup(html, "lxml")
    # 댓글이 들어있는 페이지 전체 크롤링
    comments_raw = dom.find_all("span", {"class" : "u_cbox_contents"})
    # 댓글의 text만 뽑는다.
    comments = [comment.text for comment in comments_raw]
    return comments

# 키워드 추출

In [29]:
from krwordrank.sentence import summarize_with_sentences
def keyword_word(text):#키워드 단어 추출
    data = text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=10, num_keysents=10)
    return keywords

In [30]:
def keyword_sentence(url):#키워드 문장 추출
    m_text = get_text(url)
    r_text = clean_text(m_text)
    data = r_text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=10, num_keysents=10)
    return sents

# 모델 로딩

In [31]:
from tensorflow.keras.models import load_model
# from konlpy.tag import Okt  
okt = Okt()

In [32]:
def model_load(comments):
    token_sent=[]
    for comment in comments:
        temp = okt.morphs(comment, stem=True)
        token_sent.append(temp)
    max_words = 35000
    tokenizer = Tokenizer(num_words = max_words) # 상위 35,000개의 단어만 보존
    tokenizer.fit_on_texts(token_sent) 
    token_sent = tokenizer.texts_to_sequences(token_sent)
    word_to_index = tokenizer.word_index
    vocab_size = len(word_to_index)+1
    max_len = 124
    X_data = pad_sequences(token_sent, maxlen=max_len)
    model = load_model('model11.h5')
    predict = model.predict_classes(X_data)
    a = [1 for _ in range(len(X_data))]
    for i in range(len(X_data)):
        if(predict[i] == 0):
            a[i]=0
    return a

# 플라스크

In [33]:
from flask import Flask, render_template
from flask import request, send_file, url_for
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os
# import numpy as np

In [34]:
app = Flask(__name__)
url = ''

In [35]:
class ArticleForm(Form):
    articleurl = TextAreaField('',[validators.DataRequired()])

In [36]:
@app.route('/')
def index():
    form = ArticleForm(request.form)
    return render_template('article_app.html', form=form)

In [37]:
@app.route('/title', methods = ['POST'])
def article():
    form = ArticleForm(request.form)
    if request.method == 'POST' and form.validate():
        global url
        url = request.form['articleurl']
        title = get_title(url)
        r_title = clean_text(title)
        m_text = get_text(url)
        r_text = clean_text(m_text)
        return render_template('title.html', title=r_title, content=r_text)
    return render_template('article_app.html', form=form)

In [38]:
@app.route('/result')
def review():
    global url_for
    key_sentence = keyword_sentence(url)
    cr_comment = comment(url)
    result = model_load(cr_comment)
    return render_template('result.html',list=key_sentence, list2=cr_comment, list3=result)

In [39]:
@app.route('/thanks', methods = ['POST'])
def feedback():
    feedback = request.form['feedback_button']
    return render_template('thanks.html')

In [40]:
if __name__ == '__main__':
    app.run()

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Mar/2020 00:32:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2020 00:32:59] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2020 00:32:59] "GET /static/reset.css HTTP/1.1" 200 -
